### Week 1: Excercise
- To demonstrate your familiarity with OpenAI API, and also Ollama
- Build a tool that takes a technical question and responds with an explanation. 
- This is a tool that you will be able to use yourself during the course!

In [1]:
# Import required libraries
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# Define constants for Model to be used
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv()
# Initialize OpenAI client
openai = OpenAI()

In [4]:
# here is the question; type over this to ask something new
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# ALeternative way to get user input
question = input("Please enter your question:")

In [5]:
# define system and user prompts
system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [6]:
# construct api messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

### Example - Using Gpt-4o-mini

In [7]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Certainly! Let's break down the line of code you provided:

python
yield from {book.get("author") for book in books if book.get("author")}


### Understanding the Components

1. **Set Comprehension**:
   - `{... for book in books if book.get("author")}`: This is a set comprehension, which is used to create a set. 
   - It iterates over a collection called `books`, which is expected to be a list or another iterable containing dictionaries.
   - For each `book` in `books`, it checks the condition `if book.get("author")`. This means it tries to get the value associated with the key `"author"` from the `book` dictionary. If this value is found (and is considered "truthy" - not `None`, `False`, `0`, etc.), it will include the author's name in the set. If the author does not exist, it will not include anything for that book.

2. **Set Construction**:
   - The result of the set comprehension is a set of authors for books that have an author listed. Using a set automatically handles duplicates, so if multiple books have the same author, that author will only appear once in the resulting set.

3. **Yielding from a Set**:
   - `yield from ...`: This is a Python generator expression that allows the function to yield each item from the given iterable (in this case, the set of authors). This means that the code will yield each unique author's name one by one, as the consumer of this generator iterates over it.

### Putting It All Together

In summary, this line of code does the following:

- It creates a **set** of unique authors from a list of books (which are presumably dictionaries) by extracting the `"author"` key from each book.
- It filters out any books that do not have an author (i.e., where `book.get("author")` returns a falsy value).
- Finally, it uses `yield from` to yield each unique author one at a time through a generator. This can be useful in an asynchronous context or when processing a large dataset where you want to avoid loading everything into memory at once.

### Why Use This Pattern?

This code pattern is efficient and concise for several reasons:

- **Avoids Duplicates**: By using a set, it automatically filters out any duplicated authors.
- **Lazy Evaluation**: With `yield from`, the authors are generated on-the-fly as needed rather than all at once, which is memory efficient for large datasets.
- **Readable**: The use of set comprehension simplifies code readability by clearly expressing the intention to create a set based on a condition.

### Example Scenario

Suppose you have the following list of books:

python
books = [
    {"title": "Python Programming", "author": "John Smith"},
    {"title": "Learning Python", "author": "Jane Doe"},
    {"title": "Advanced Python", "author": None},
    {"title": "Python for Data Science", "author": "John Smith"},
    {"title": "Data Analysis", "author": "Jane Doe"},
]


Running the code would yield the following unique authors:

python
# Yields: "John Smith", "Jane Doe"


### Conclusion

This code effectively extracts unique authors from a collection of books while handling cases where authors may be missing, using set comprehension for clarity and efficiency, and yielding results in a memory-efficient manner with a generator.

### Example - Using Llama3.2 (local docker env.)

In [8]:
# Get Llama 3.2 to answer
response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

**Deconstruction and Generator Expression**

The given code is written in Python 3.5 and later versions, which supports a feature called "deconstruction" and "generator expressions." Let's break it down:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's what happens step by step:

1. **List Comprehension**: `{book.get("author") for book in books if book.get("author")}` is a generator expression, which generates a sequence of values on-the-fly.

   * The `for` loop iterates over the `books` collection ( likely a list or dictionary of dictionaries).
   * For each iteration, it checks if the current item (`book`) has an attribute named `"author"`. If it does, it extracts the value associated with this key.
   * The resulting values are then collected into a set.

2. **`yield from`**: This is a special syntax used in Python 3.3 and later versions to delegate the iteration over a subiterator (in this case, the generator expression) back to the caller.

   When `yield from` encounters an iterator that yields values, it simply "yields" those same values in the current context, effectively passing on the responsibility of generating the next value.

   In our example, `{book.get("author") for book in books if book.get("author")}` is a generator expression. The `yield from` statement takes this generator expression and uses its values without changing the flow of execution.

**Effect**

In summary, the given code:

* Iterates over the `books` collection.
* For each book that has an `"author"` attribute, it extracts and yields the associated value in a set.
* The `yield from` statement takes this generator expression and uses its values without changing the flow of execution.

This is equivalent to writing:
```python
result = set()
for book in books:
    if book.get("author"):
        result.add(book.get("author"))
yield from result
```
But with a more concise and efficient syntax!

**Example Use Case**

Here's an example where this code might be used:

Suppose you have a list of book dictionaries, each containing author information. You want to extract all the unique authors from these books. The given code can be used in a context like this:
```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},  # No author for Book 3
]

authors = {book.get("author") for book in books if book.get("author")}
print(authors)  # Output: {'Author A', 'Author B'}
```
In this case, the code extracts all unique authors from the `books` list and stores them in a set named `authors`.